<a href="https://colab.research.google.com/github/SunSlick2/MoveEmails/blob/main/Online_Archive_Root_to_Inbox_Migrator_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import win32com.client
import pythoncom
import logging
import time
from datetime import datetime
import json
import os
import tkinter as tk
from tkinter import messagebox

class EmailMigrator:
    def __init__(self):
        self.setup_logging()
        self.migration_report = {
            'start_time': None,
            'end_time': None,
            'source_store': 'N/A',
            'source_folder': 'N/A',
            'destination_folder': 'N/A',
            'total_items_found': 0,
            'total_successful': 0,
            'total_failed': 0
        }

    def setup_logging(self):
        """Setup comprehensive logging for the migration."""
        log_dir = "migration_logs"
        os.makedirs(log_dir, exist_ok=True)
        log_filename = os.path.join(log_dir, f'online_archive_migration_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger().addHandler(console)

        file_handler = logging.FileHandler(log_filename)
        file_handler.setLevel(logging.DEBUG)
        file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logging.getLogger().addHandler(file_handler)

        logging.info(f"Logging to: {log_filename}")

    def find_store_by_display_name(self, namespace, store_name):
        """
        Finds an Outlook store (mailbox or PST) by its display name.
        """
        for store in namespace.Stores:
            if store.DisplayName == store_name:
                logging.info(f"Store found: '{store.DisplayName}'")
                return store
        return None

    def migrate_items(self, source_folder, destination_folder):
        """Moves all items from a source folder to a destination folder with throttling."""
        moved_count = 0
        failed_count = 0

        # --- Throttling Configuration ---
        THROTTLE_LIMIT = 50  # Move 50 items
        THROTTLE_DELAY = 1   # Then wait for 1 second

        try:
            items_list = list(source_folder.Items)
            self.migration_report['total_items_found'] = len(items_list)

            logging.info(f"Found {len(items_list)} items to move.")

            for i, item in enumerate(items_list):
                try:
                    item.Move(destination_folder)
                    moved_count += 1

                    # Apply throttling
                    if (i + 1) % THROTTLE_LIMIT == 0:
                        logging.info(f"Pausing for {THROTTLE_DELAY} seconds to respect server limits...")
                        time.sleep(THROTTLE_DELAY)

                    logging.info(f"Moved item: '{getattr(item, 'Subject', 'N/A')}'")
                except Exception as e:
                    failed_count += 1
                    logging.warning(f"Failed to move item '{getattr(item, 'Subject', 'N/A')}': {e}")
        except Exception as e:
            logging.error(f"Error accessing items in source folder: {e}", exc_info=True)
            failed_count = self.migration_report['total_items_found']

        self.migration_report['total_successful'] = moved_count
        self.migration_report['total_failed'] = failed_count

        return moved_count, failed_count

    def run_migration(self):
        """Main function to run the migration process."""
        self.migration_report['start_time'] = datetime.now().isoformat()

        # --- Hardcoded Paths ---
        ONLINE_ARCHIVE_NAME = "Online Archive - ghi.jkl@def.com"

        pythoncom.CoInitialize()
        outlook = None

        try:
            outlook = win32com.client.Dispatch("Outlook.Application")
            namespace = outlook.GetNamespace("MAPI")

            # Step 1: Find the online archive store
            logging.info(f"Finding online archive: '{ONLINE_ARCHIVE_NAME}'")
            archive_store = self.find_store_by_display_name(namespace, ONLINE_ARCHIVE_NAME)
            if not archive_store:
                raise ValueError(f"Online Archive '{ONLINE_ARCHIVE_NAME}' not found. Please ensure it is open in Outlook.")

            self.migration_report['source_store'] = archive_store.DisplayName

            # Step 2: Get the source and destination folders
            source_folder = archive_store.GetRootFolder()

            # Ensure Inbox exists before attempting to access it
            try:
                destination_folder = source_folder.Folders['Inbox']
            except Exception:
                raise ValueError("Inbox folder not found in the Online Archive root. Please ensure the standard Inbox folder structure exists.")


            self.migration_report['source_folder'] = source_folder.FolderPath
            self.migration_report['destination_folder'] = destination_folder.FolderPath

            print("\n" + "=" * 50)
            print("CONFIRMATION: You are about to MOVE ALL emails from:")
            print(f"Source: {self.migration_report['source_folder']}")
            print(f"To Destination: {self.migration_report['destination_folder']}")
            print("\nThis action will MOVE items and CANNOT BE UNDONE.")
            print("The script is now throttled to prevent server errors.")
            print("=" * 50)

            confirm = input("Type 'CONFIRM' to proceed with the migration: ")
            if confirm.strip().upper() != 'CONFIRM':
                logging.warning("Migration cancelled by user.")
                print("Operation cancelled.")
                return False

            # Step 3: Start the migration
            self.migrate_items(source_folder, destination_folder)

            # Step 4: Finalize and report
            self.migration_report['end_time'] = datetime.now().isoformat()
            self.generate_report()

            if self.migration_report['total_failed'] == 0:
                logging.info("Migration completed successfully!")
                return True
            else:
                logging.warning(f"Migration completed with {self.migration_report['total_failed']} errors. Check the log file for details.")
                return False

        except Exception as e:
            logging.error(f"Critical error during migration: {e}", exc_info=True)
            messagebox.showerror("Migration Error", f"A critical error occurred. Check logs for details: {e}")
            return False
        finally:
            if outlook:
                del outlook
            pythoncom.CoUninitialize()

    def generate_report(self):
        """Generates a comprehensive report of the migration."""
        report_dir = "migration_reports"
        os.makedirs(report_dir, exist_ok=True)
        report_filename = os.path.join(report_dir, f'migration_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')

        with open(report_filename, 'w') as f:
            json.dump(self.migration_report, f, indent=2, default=str)

        logging.info(f"Migration report saved to: {report_filename}")

        print("\n=== MIGRATION SUMMARY ===")
        print(f"Source Folder: {self.migration_report['source_folder']}")
        print(f"Destination Folder: {self.migration_report['destination_folder']}")
        print(f"Total Items Found: {self.migration_report['total_items_found']}")
        print(f"Total Items Moved: {self.migration_report['total_successful']}")
        print(f"Total Items Failed: {self.migration_report['total_failed']}")

# Usage
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("  Online Archive Root to Inbox Migrator   ")
    print("=" * 60)
    print("This tool will move all emails from your Online Archive's")
    print("root folder to its Inbox folder.")
    print("\n" + "=" * 60)

    migrator = EmailMigrator()
    migrator.run_migration()